In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/training_set_rel3.tsv", sep='\t', on_bad_lines='skip', encoding='ISO-8859-1')
df.dropna(axis=1,inplace=True)
df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
df.head(100)



,essay_id,essay_set,essay
0,1,1,"Dear local newspaper, I think effects computer..."
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu..."
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl..."
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that..."
4,5,1,"Dear @LOCATION1, I know having computers has a..."
...,...,...,...
95,96,1,The use of computers is definetly correct. It ...
96,97,1,"Dear Local @CAPS1, I think computers benefit s..."
97,98,1,"Dear @ORGANIZATION1, In this new digital age, ..."
98,99,1,Dear @CAPS1: Have you been sent a photo of a @...


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Processed_data.csv")
data.drop("Unnamed: 0",inplace=True,axis=1)

In [ ]:
df['domain1_score']=data['final_score']
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",6
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",5
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",8
4,5,1,"Dear @LOCATION1, I know having computers has a...",6


In [ ]:
data['essay'][0]


"Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the economy, sea floor spreading or even about the

In [ ]:
data.head(1)

,essay_id,essay_set,essay,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",6,Dear local newspaper I think effects computer...,1441,344,16,4.188953,11,76,75,18,24


In [ ]:
#Make Dataset
y = data['final_score']
# df.drop('domain1_score',inplace=True,axis=1)
X=data



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape

(9083, 14)

In [ ]:
X_train.head()

,essay_id,essay_set,essay,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
12481,21003,8,It's the first day of high school and your gut...,6,It s first day high school gut full butterflie...,2310,595,31,3.882353,17,119,135,35,29
6425,9982,4,The author concludes the story with that parag...,7,The author concludes story paragraph many reas...,454,101,7,4.495050,6,25,19,8,2
4989,7388,3,Many things that the author talks about in the...,7,Many things author talks story affect One exa...,578,139,8,4.158273,3,43,22,6,1
12903,21536,8,"Have you ever had a memorable time, where you ...",6,Have ever memorable time laughed hard friends...,2049,517,29,3.963250,10,95,97,42,47
10239,16189,6,As a result of the idea to build a mooring mas...,8,As result idea build mooring mast builder fac...,752,160,9,4.700000,12,45,29,14,5


In [ ]:
y_train.head()

12481    6
6425     7
4989     7
12903    6
10239    8
Name: final_score, dtype: int64

# ***PREPROCESSING OF THE DATASETS***


In [ ]:
train_e = X_train['clean_essay'].tolist() #Essays are coverted in the list format
test_e = X_test['clean_essay'].tolist()

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english')) 
def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)

In [ ]:
train_sents[0]

['It',
 'first',
 'day',
 'high',
 'school',
 'gut',
 'full',
 'butterflies',
 'make',
 'want',
 'run',
 'bathrooms',
 'hide',
 'world',
 'I',
 'friends',
 'yet',
 'I',
 'shy',
 'go',
 'join',
 'groups',
 'I',
 'walked',
 'classes',
 'alone',
 'Students',
 'would',
 'get',
 'face',
 'wanting',
 'know',
 'school',
 'I',
 'came',
 'wanted',
 'know',
 'I',
 'remember',
 'wanting',
 'left',
 'alone',
 'hidden',
 'bubble',
 'space',
 'Girls',
 'would',
 'ask',
 'I',
 'upset',
 'since',
 'I',
 'nt',
 'smiling',
 'fun',
 'getting',
 'know',
 'new',
 'classes',
 'Lunch',
 'much',
 'crowded',
 'old',
 'school',
 'Students',
 'would',
 'sit',
 'hallways',
 'eating',
 'lunch',
 'crowd',
 'around',
 'one',
 'big',
 'table',
 'chattering',
 'loudly',
 'one',
 'another',
 'I',
 'chewed',
 'lip',
 'looking',
 'spot',
 'nt',
 'occupied',
 'students',
 'I',
 'found',
 'table',
 'near',
 'front',
 'lunch',
 'room',
 'empty',
 'placed',
 'brown',
 'lunch',
 'sack',
 'front',
 'eating',
 'peanut',
 'butte

# **LSTM MODEL**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding
from tensorflow.keras.optimizers import Adam

In [ ]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='relu'))
    # model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error', metrics=['mse', 'mae'])
    model.summary()
    return model

# **WORD2VEC MODEL**

In [ ]:
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

model = Word2Vec(train_sents, 
                 workers=num_workers, 
                 vector_size=num_features,   #new parameter name vector_size
                 min_count = min_word_count, 
                 window = context, 
                 sample = downsampling)


model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

In [ ]:
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.wv.index_to_key)   #new attribute index_to_key
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec,model.wv[i])   #word fetch in model
            np.seterr(invalid='ignore')

    vec = np.divide(vec,noOfWords)
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs


clean_train=[]
for i in train_e:
    clean_train.append(sent2word(i))
training_vectors = getVecs(clean_train, model, num_features)

clean_test=[] 

for i in test_e:
    clean_test.append(sent2word(i))
testing_vectors = getVecs(clean_test, model, num_features)

In [ ]:
training_vectors.shape

(9083, 300)

In [ ]:
training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 300)            721200    
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 814,770
Trainable params: 814,770
Non-trainable params: 0
_________________________________________________________________


In [ ]:
training_vectors.shape

(9083, 1, 300)

In [ ]:
lstm_model.fit(training_vectors, y_train, batch_size=40, epochs=50)

Epoch 1/50
228/228 [==============================] - 6s 27ms/step - loss: 1.2538 - mse: 1.2538 - mae: 0.8577
Epoch 2/50
228/228 [==============================] - 9s 41ms/step - loss: 1.2083 - mse: 1.2083 - mae: 0.8492
Epoch 3/50
228/228 [==============================] - 11s 49ms/step - loss: 1.2076 - mse: 1.2076 - mae: 0.8453
Epoch 4/50
228/228 [==============================] - 12s 51ms/step - loss: 1.2164 - mse: 1.2164 - mae: 0.8457
Epoch 5/50
228/228 [==============================] - 8s 37ms/step - loss: 1.1813 - mse: 1.1813 - mae: 0.8385
Epoch 6/50
228/228 [==============================] - 7s 30ms/step - loss: 1.1916 - mse: 1.1916 - mae: 0.8442
Epoch 7/50
228/228 [==============================] - 6s 26ms/step - loss: 1.1679 - mse: 1.1679 - mae: 0.8296
Epoch 8/50
228/228 [==============================] - 5s 23ms/step - loss: 1.1909 - mse: 1.1909 - mae: 0.8436
Epoch 9/50
228/228 [==============================] - 7s 31ms/step - loss: 1.2098 - mse: 1.2098 - mae: 0.8475
Epoch 10

In [ ]:
lstm_model.save('final_lstm.h5')
y_pred = lstm_model.predict(testing_vectors)
y_pred = np.around(y_pred)
y_pred

122/122 [==============================] - 1s 7ms/step


array([[4., 4.],
       [8., 8.],
       [7., 7.],
       ...,
       [8., 8.],
       [7., 7.],
       [9., 9.]], dtype=float32)